----
# 기본적인 뉴럴 네트워크
### 의류 판별 MNNIST


In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms



In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            torch.nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            torch.nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            torch.nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 64),
            torch.nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


* FashionMNIST 데이터는 28x28 크기의 흑백 이미지 데이터 입니다. 이 입력에 대해 Flatten 계층에서 각 28x28의 2D 이미지를 784 픽셀 값을 갖는 연속된 배열로 변환합니다.
* 최종적으로 10개의 확률적 출력을 만들어 냅니다.


In [ ]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


torchvision 의 datasets에서 FashionMNIST 데이터를 가져와서 torchvision 의 transforms.ToTensor() 로 텐서로 변환 합니다. transform 으로 ToTensor() 함수를 사용하는 것은 PIL 이미지 또는 numpy.ndarray를 pytorch의 텐서로 변형시켜 줍니다. 그다음 batch_size=64 인 DataLoader 를 만들어 줍니다.


In [ ]:
model = NeuralNetwork()
model.to(device)
print(model)


학습에 필요한 하이퍼 파라메터를 지정합니다.


In [ ]:
learning_rate = 1e-3
batch_size = 32
epochs = 30


손실함수와 옵티마이저를 정의 합니다.


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


학습 loop 를 정의 합니다. 학습 단계(loop)에서 최적화는 세단계로 이뤄집니다:

1. optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정합니다. 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정합니다.

2. loss.backward()를 호출하여 예측 손실(prediction loss)을 역전파합니다. PyTorch는 각 매개변수에 대한 손실의 변화도를 저장합니다.

3. 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정합니다.


In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # 예측(prediction)과 손실(loss) 계산
        X, y = X.to(device), y.to(device)  # 데이터를 디바이스로 이동
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)  # 데이터를 디바이스로 이동
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


epoch 수만큼 train loop 를 수행하여 반복 학습 합니다.


In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


6x6=36 개의 이미지를 테스트 해봅니다.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 클래스 레이블에 대응하는 태그
label_tags = {
    0: 'T-Shirt',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle Boot'
}

columns = 6
rows = 6

fig = plt.figure(figsize=(15, 15))

# batch size와 다른사이즈(ex:1개)를 테스트 할경우에는 eval 호출 필요
model.eval()  # 모델을 평가 모드로 변경

for i in range(1, columns * rows + 1):
    data_idx = np.random.randint(len(test_dataloader))
    input_img, label = test_data[data_idx][0].to(device), test_data[data_idx][1]

    # 이미지 예측
    with torch.no_grad():
        output = model(input_img)
        _, prediction = torch.max(output, 1)
        pred_class = label_tags[prediction.item()]
        true_class = label_tags[label]

    # 제목과 색상 설정
    title = f'{pred_class}, right !' if pred_class == true_class else f'Not {pred_class} but {true_class}'
    cmap = 'Blues' if pred_class == true_class else 'Reds'

    fig.add_subplot(rows, columns, i)
    plot_img = input_img.squeeze().cpu().numpy()  # 텐서를 넘파이 배열로 변환하여 이미지 플로팅
    plt.imshow(plot_img, cmap=cmap)
    plt.title(title)
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
a = torch.randn(4, 4)
a

In [ ]:
torch.max(a, 1)

In [ ]:
torch.max(a)

In [ ]:
torch.max(a, -1)

In [ ]:
test_data[data_idx][0].shape

In [ ]:
input_img.squeeze().shape

In [ ]:
output = model(input_img)
print(output)

In [ ]:
torch.max(output, 1)

### 숫자 판별 MNNIST

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# 데이터셋 로드 및 전처리
transform = transforms.Compose([
    transforms.ToTensor(),  # 이미지를 텐서로 변환
    transforms.Normalize((0.5,), (0.5,))  # 평균과 표준편차를 사용하여 정규화
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

# 신경망 정의
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        ############### Your code here!!!
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            torch.nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Linear(512, 256),
            torch.nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Linear(256, 128),
            torch.nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128, 64),
            torch.nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64, 10),
        )

    def forward(self, x):
        ############### Your code here!!!
        # 활성화 함수는 relu 사용
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)  # 데이터를 디바이스로 이동
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")



model = SimpleNN()

############### Your code here!!!
# 손실 함수와 옵티마이저 설정
#criterion = ??
#optimizer = ??
learning_rate = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 학습
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

# num_epochs = ??
num_epochs = 10
a = 0
l = []
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print(f"[Epoch {epoch + 1}, Batch {i + 1}] Loss: {running_loss / 100:.3f}")
            l.append(running_loss / 100)
            running_loss = 0.0

print("Finished Training")

# 모델 저장
torch.save(model.state_dict(), 'mnist_simple_nn.pth')


In [ ]:
import matplotlib.pyplot as plt

# 학습 완료 후 테스트 데이터로 예측 수행
model.eval()
test_images, test_labels = next(iter(testloader))
test_images, test_labels = test_images.to(device), test_labels.to(device)
with torch.no_grad():
    test_outputs = model(test_images)
softmax = nn.Softmax(dim=1)
probabilities = softmax(test_outputs)
predicted_classes = torch.argmax(probabilities, dim=1)

# 이미지 및 예측 시각화
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[i].cpu().numpy().squeeze(), cmap=plt.cm.binary)
    plt.xlabel(f'Actual: {test_labels[i]}, Predicted: {predicted_classes[i]}')
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# 선 그래프 그리기
plt.plot(l)

# 그래프에 제목 추가
plt.title('Loss Graph')

# x축과 y축에 레이블 추가
plt.xlabel('Number')
plt.ylabel('Loss')

# 그래프 표시
plt.show()